# Sommaire

## Introduction

> Il s’agit de développer une solution de scoring permettant d’apporter une aide à la décision pour les chargés de relation client.
>
> En effet, basée sur des algorithmes de Machine Learning, la solution calcule la probabilité qu’un client rembourse ou non son crédit.

## Ressources

> Pour réaliser votre projet, votre manager, vous a fourni le jeu de données suivant qui contient :
- Un historique de prêts
- Un historique d’informations financières
- Des informations sur le comportement des emprunteurs (si l’emprunteur a fait défaut ou pas).

## Attentes et livrables

> Il vous a également communiqué ses attentes par rapport votre projet :
- Ce premier projet autour de l’IA est un bon moyen pour sensibiliser l’équipe à ce domaine. Un meetup sur ce sujet de l’IA et la Finance sera le bienvenue
- Un premier POC doit être rapidement réalisé en vue de permettre à nos chargés de relation de tester rapidement la solution
- Le modèle doit être interprétable par les équipes qui vont l’utiliser
- Votre travail doit être facilement réutilisable par les futurs data scientists qui vont rejoindre l’équipe
- Votre approche de la modélisation de la problématique ainsi que votre méthode de gestion de ce projet doivent être présentées à l’équipe

> Les livrables attendus :
- Un Jupyter Notebook avec un premier tuto d’apprentissage (PIMA, Iris, etc.)
- Un Jupyter Notebook présentant les différentes parties de votre travail de modélisation
- Un meetup sur le sujet (option pour la veille)
- Une solution déployée
- Une présentation (PowerPoint)

## Critère d'évaluation

|Compétences|Critère d'évaluation|
|-|-|
|Etudes théorique|4 modèles de classifications sont étudiés|
|Réalisation de feature engineering|Analyse Exploratoire|
|&nbsp;|Au moins 2 approches de feature engineering sont testés dont Boruta|
|&nbsp;|Les hypothèses retenues pour la sélection des variables sont justifiées|
|Entrainement du modèle|Les modèles ML et les métriques d’évaluation sont choisis en fonction de la problématique métier|
|&nbsp;|Au moins 3 modèles ML sont testés|
|&nbsp;|Les hyperparamètres des modèles sont identifiés et optimisés (GridSearch)|
|&nbsp;|La validation croisé est utilisée|
|&nbsp;|Une analyse comparative des modèles est réalisée|
|Prise en compte des enjeux de l’IA|Des approches d’interprétabilité des modèles ML sont testées|
|&nbsp;|Les résultats du modèle de scoring doivent être présentés à un public non technique (chargés de relation)|
|Déploiement de la solution|La solution développée  est mise à disposition de l’utilisateur final|
|Qualité des livrables|Un Jupyter Notebook présentant l’ensemble de la démarche|
|&nbsp;|Une solution qui répond aux besoins de l’utilisateur final|
|&nbsp;|Un support avec la présentation de la problématique et la solution proposée|